In [1]:
import numpy as np
import pandas as pd
import rasterio
import rioxarray as rxr
import xarray as xr
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
rxr.open_rasterio('./Vegetation HDF/MOD13A2.A2020337.h20v05.061.2021011170632.hdf')

<xarray.Dataset> Size: 33MB
Dimensions:                                 (band: 1, x: 1200, y: 1200)
Coordinates:
  * band                                    (band) int64 8B 1
  * x                                       (x) float64 10kB 2.224e+06 ... 3....
  * y                                       (y) float64 10kB 4.447e+06 ... 3....
    spatial_ref                             int64 8B 0
Data variables:
    1 km 16 days NDVI                       (band, y, x) int16 3MB ...
    1 km 16 days relative azimuth angle     (band, y, x) int16 3MB ...
    1 km 16 days composite day of the year  (band, y, x) int16 3MB ...
    1 km 16 days pixel reliability          (band, y, x) int8 1MB ...
    1 km 16 days EVI                        (band, y, x) int16 3MB ...
    1 km 16 days VI Quality                 (band, y, x) uint16 3MB ...
    1 km 16 days red reflectance            (band, y, x) int16 3MB ...
    1 km 16 days NIR reflectance            (band, y, x) int16 3MB ...
    1 km 16 days blue reflectance           (band, y, x) int16 3MB ...
    1 km 16 days MIR reflectance            (band, y, x) int16 3MB ...
    1 km 16 days view zenith angle          (band, y, x) int16 3MB ...
    1 km 16 days sun zenith angle           (band, y, x) int16 3MB ...
Attributes: (12/173)
    ALGORITHMPACKAGEACCEPTANCEDATE:      102004
    ALGORITHMPACKAGEMATURITYCODE:        Normal
    ALGORITHMPACKAGENAME:                MOD_PR13A1
    ALGORITHMPACKAGEVERSION:             6
    ASSOCIATEDINSTRUMENTSHORTNAME.1:     MODIS
    ASSOCIATEDPLATFORMSHORTNAME.1:       Terra
    ...                                  ...
    SOUTHBOUNDINGCOORDINATE:             29.9999999973059
    SPSOPARAMETERS:                      2749, 4334, 2749a, 4334a
    TileID:                              51020005
    VERSIONID:                           61
    VERTICALTILENUMBER:                  5
    WESTBOUNDINGCOORDINATE:              23.0940107648842

In [3]:
date = pd.to_datetime('01-01-2020')
date

Timestamp('2020-01-01 00:00:00')

In [4]:
def read_land_cover(hdf_path, var_name, new_name):
    # Read as DataArrays with a "band" dimension of length up to 4
    aod = rxr.open_rasterio(hdf_path)[var_name]  * .0001
    #qa  = rxr.open_rasterio(sds(hdf_path, "AOD_QA"),             masked=True).astype("uint16")

    # Scale AOD (INT16 * 0.001). Keep shape [band, y, x].
    #aod = aod * 0.001

    # Apply QA per band
    #good = mask_from_qa(qa.values)  # shape [band, y, x]
    #aod.data = np.where(good, aod.values, np.nan)

    # Reduce across multiple overpasses (bands) to a single daily raster (median is robust)
    # (xarray names the dim "band" by default)
    #aod_daily = aod.median(dim="band")#, skipna=True)

    # Ensure NoData is NaN
    aod_daily = aod.rename(new_name)
    #aod_daily.rio.write_nodata(np.nan, inplace=True)
    return aod_daily

In [5]:
def mosaic_and_reproject(list_of_dss):
    # Mosaic in native CRS
    '''
    Mosaic multiple DataArrays by averaging overlapping pixels.
    Assumes all DataArrays have the same CRS and resolution.
    list_if_dss: list of xarray.DataArray
    returns: xarray.DataArray
    '''
    stack = xr.concat(list_of_dss, dim="stack")
    mosaic = stack.mean("stack", skipna=True)

    # Reproject to WGS84 (≈1 km at equator = 30 arcsec)
    mosaic_wgs84 = mosaic.rio.reproject(
        "EPSG:4326",
        resolution=0.01, #0.00926625433
        resampling=rasterio.enums.Resampling.nearest,
        nodata=np.nan
    )
    return mosaic_wgs84

In [6]:
#ok
def convert_names_to_dates(files):
    year = []
    day = []
    #hr = []
    #min = []
    dts = []
    for file in files:
        dts.append(file[9:16])
        year.append(int(file[9:13]))
        day.append(int(file[13:16]))
        #hr.append(int(file[18:20]))
        #min.append(int(file[20:22]))
    files = pd.DataFrame(files, columns=['Files'])
    dts = pd.DataFrame(dts, columns=['MODIS_Dates'])
    year = pd.DataFrame(year, columns=['Year'])
    day = pd.DataFrame(day, columns=['Day'])
    #hr = pd.DataFrame(hr, columns=['Hour'])
    #min = pd.DataFrame(min, columns=['Minute'])
    dtdf = pd.concat([files, dts, year, day], axis=1)
    dtdf['datetime'] = pd.to_datetime(dtdf['Year'] * 1000 + dtdf['Day'], format='%Y%j')
    dtdf.sort_values(by=['datetime'], inplace=True, ascending=True)
    dtdf.reset_index(drop=True, inplace=True)
    return dtdf

In [7]:
convert_names_to_dates(os.listdir('./Vegetation HDF/'))['datetime'][0] < convert_names_to_dates(os.listdir('./Vegetation HDF/'))['datetime'][5]

True

In [8]:
def hdf_to_netcdf(folder, minx, miny, maxx, maxy, var_name, new_name):
    files = os.listdir(folder)
    df = convert_names_to_dates(files)
    for date in df['datetime'].unique():
        dff = df[df['datetime'] == date]
        dss = [read_land_cover(folder + file, var_name, new_name) for file in dff['Files'] if file.endswith('.hdf')]
        s = mosaic_and_reproject(dss).rio.clip_box(minx=minx-1e-16, miny=miny-1e-16, maxx=maxx+1e-16, maxy=maxy+1e-16)
        s['x'] = np.round(s['x'], 2)
        s['y'] = np.round(s['y'], 2)
        ds_date = s.to_dataset().expand_dims(time=[date]).assign_coords(time=("time", [date]))
        if date == df['datetime'].unique()[0]:
            final_ds = ds_date
        else:
            final_ds = xr.concat([final_ds, ds_date], dim='time')

    return final_ds

In [9]:
minx=35.5
miny=31.0
maxx=37.4
maxy=32.2

In [10]:
folder = './Vegetation HDF/'

In [11]:
ndvi = hdf_to_netcdf(folder, minx, miny, maxx, maxy, '1 km 16 days NDVI', 'NDVI')
evi = hdf_to_netcdf(folder, minx, miny, maxx, maxy, '1 km 16 days NDVI', 'EVI')

In [12]:
ndvi

<xarray.Dataset> Size: 18MB
Dimensions:      (time: 96, band: 1, y: 121, x: 191)
Coordinates:
  * time         (time) datetime64[ns] 768B 2020-12-02 2020-12-18 ... 2025-01-17
  * band         (band) int64 8B 1
  * y            (y) float64 968B 32.2 32.19 32.18 32.17 ... 31.02 31.01 31.0
  * x            (x) float64 2kB 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
    spatial_ref  int64 8B 0
Data variables:
    NDVI         (time, band, y, x) float64 18MB 0.2117 0.2326 ... 0.1287 0.1298

In [13]:
#nc = hdf_to_netcdf(folder, minx, miny, maxx, maxy)
def process(nc):
    df = nc.to_dataframe().reset_index().drop(['band', 'spatial_ref'], axis=1)
    df['latitude'] = df['y']
    df['longitude'] = df['x']
    df.drop(['x', 'y'], axis=1, inplace=True)
    df.set_index(['time', 'latitude', 'longitude'], inplace=True)
    #df['LC'] = np.int8(df['LC'])
    nc = df.to_xarray()
    return nc

In [14]:
ndvi = process(ndvi)
evi = process(evi)

In [15]:
def create_times(nc, start_str, end_str):
    ncs =[]
    times = []
    start = pd.to_datetime(start_str)
    end = pd.to_datetime(end_str)
    ncdts = nc['time'][(nc['time'] > start) & (nc['time'] < end)]
    for i in range(len(nc['time'])):
        if i == 0:
            t = pd.Series(pd.date_range(start=start, 
                        end=str(ncdts[i].values)[0:10], 
                        freq='D'), 
                        name='time')
            times.append(t[0:-1])
        elif i == len(nc['time'])-1:
            t = pd.Series(pd.date_range(start=str(nc['time'][i].values)[0:10], 
                        end=end, 
                        freq='D'), 
                        name='time')
            times.append(t)
        elif (i != 0) & (i != len(nc['time'])):
            t = pd.Series(pd.date_range(start=str(ncdts[i-1].values)[0:10], 
                        end=str(ncdts[i].values)[0:10], 
                        freq='D'), 
                        name='time')
            times.append(t[0:-1])

    return times



In [21]:
times = create_times(evi, '12-10-2020', '01-20-2025')

In [17]:
def fill_ds(nc, start_str, end_str):
    times = create_times(nc, start_str, end_str)
    ncs = []
    for i in range(len(times)):
        lons = nc['longitude']
        lats = nc['latitude']
        time = nc['time'][i]
        dat = nc.sel(time=time)
        arr = dat[list(dat.data_vars)[0]]
        arr_kron = np.kron(arr, np.ones((len(times[i]), 1, 1)))

        ncs.append(xr.Dataset({list(dat.data_vars)[0]:(("time","latitude" ,"longitude"), arr_kron)},
                    coords={
            "time": times[i],
            "latitude": lats,
            "longitude": lons,
            
        }))
    return xr.merge(ncs)

In [18]:
evi_full = fill_ds(evi, '12-10-2020', '20-01-2025')
ndvi_full = fill_ds(ndvi, '12-10-2020', '20-01-2025')

In [19]:
evi_full

<xarray.Dataset> Size: 278MB
Dimensions:    (latitude: 121, longitude: 191, time: 1503)
Coordinates:
  * latitude   (latitude) float64 968B 31.0 31.01 31.02 ... 32.18 32.19 32.2
  * longitude  (longitude) float64 2kB 35.5 35.51 35.52 ... 37.38 37.39 37.4
  * time       (time) datetime64[ns] 12kB 2020-12-10 2020-12-11 ... 2025-01-20
Data variables:
    EVI        (time, latitude, longitude) float64 278MB 0.1204 ... 0.1138

In [20]:
ndvi_full

<xarray.Dataset> Size: 278MB
Dimensions:    (latitude: 121, longitude: 191, time: 1503)
Coordinates:
  * latitude   (latitude) float64 968B 31.0 31.01 31.02 ... 32.18 32.19 32.2
  * longitude  (longitude) float64 2kB 35.5 35.51 35.52 ... 37.38 37.39 37.4
  * time       (time) datetime64[ns] 12kB 2020-12-10 2020-12-11 ... 2025-01-20
Data variables:
    NDVI       (time, latitude, longitude) float64 278MB 0.1204 ... 0.1138

In [25]:
dat = xr.merge([ndvi_full, evi_full])

In [26]:
# Number of chunks
n = 9

# Get indices to split along the time dimension
time_len = dat.dims["time"]
splits = np.array_split(np.arange(time_len), n)

# Save each chunk to a separate file
for i, idx in enumerate(splits):
    ds_subset = dat.isel(time=idx)
    ds_subset.to_netcdf(f"./Vegetation nc/Vegetation_{i+1}.nc")